# DNN classifier

reference data: https://labs.criteo.com/2013/12/download-terabyte-click-logs-2/

In [1]:
import pandas as pd
import numpy as np
import pickle
from os import path
from matplotlib import pyplot as plt
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import PreprocessingLayer, TextVectorization

### 1. Data batching

#### 1.1. Load dataset

In [2]:
LBL_COLUMN = ['lbl']
CAT_COLUMNS = [f'cat{i}' for i in range(26)]
NUM_COLUMNS = [f'num{i}' for i in range(13)]
COLUMNS = LBL_COLUMN + NUM_COLUMNS + CAT_COLUMNS
FEATURE_COLUMNS = NUM_COLUMNS + CAT_COLUMNS
COLUMN_DEFAULTS = [0]*14 + ['thisisdefault']*26

In [3]:
dataset = tf.data.experimental.make_csv_dataset(
    file_pattern='data/dac/sample_train.txt', 
    batch_size=200,
    num_epochs=1,
    column_defaults=COLUMN_DEFAULTS,
    column_names=COLUMNS, 
    label_name='lbl', 
    field_delim='\t',
    shuffle=True
)\
.shuffle(10, reshuffle_each_iteration=False)

#### 1.2. Conbine the columes

In [4]:
class FeaturesExtraction:
    
    def __init__(self, num_col, feature_type=None):
        self.num_col = num_col
        self.feature_type = feature_type

    def __call__(self, features, labels):
        numeric_features = [features.pop(col) for col in self.num_col]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        
        if self.feature_type == 'cat':
            return features
        if self.feature_type == 'numeric':
            return numeric_features
        if self.feature_type == 'no_label':
            features['numeric'] = numeric_features
            return features
        else:
            features['numeric'] = numeric_features
            return features, labels


In [5]:
packed_dataset = dataset.map(FeaturesExtraction(num_col=NUM_COLUMNS))

#### 1.3. Training / validation split

In [6]:
def is_validate(idx, data):
    return idx % 5 == 0

def is_train(idx, data):
    return not is_validate(idx, data)

recover = lambda idx, data: data

validate_dataset = packed_dataset.enumerate()\
.filter(is_validate)\
.map(recover)

train_dataset = packed_dataset.enumerate()\
.filter(is_train)\
.map(recover)

### 2. Data processing

In [7]:
class DataProcessingLayer(PreprocessingLayer):
    
    def __init__(self, ls_cat_col, num_col, vocabulary=None, **kwargs):
        
        super(DataProcessingLayer, self).__init__(**kwargs)
        self._ls_cat_col = ls_cat_col
        self._num_col = num_col
        self._dict_cat_vocab = vocabulary
        
        self._dict_vectorization_layer = dict()
        for key in ls_cat_col:
            self._dict_vectorization_layer.update({
                key: TextVectorization(output_sequence_length=1)
            })
        self.processing_layer = None
    
    def adapt(self, data):
        
        for cat_col in self._ls_cat_col:
            print(f'adapting col: {cat_col}')
            tmp_dataset = data.map(lambda feature, label: feature.pop(cat_col))
            self._dict_vectorization_layer[cat_col].adapt(tmp_dataset)
        
        self._dict_cat_vocab = dict()
        for cat_col in self._ls_cat_col:
            self._dict_cat_vocab.update({
                cat_col: self._dict_vectorization_layer[cat_col].get_vocabulary()
            })
                
    def build(self, input_shape):
        
        ls_feature_cat_col = list()
        ls_feature_num_col = list()
        
        for cat_col, vocab in self._dict_cat_vocab.items():
            tf_cat_vocab = tf.feature_column.categorical_column_with_vocabulary_list(
                key=cat_col, vocabulary_list=vocab)
            ls_feature_cat_col.append(tf.feature_column.indicator_column(tf_cat_vocab))
        
        tf_feature_num = tf.feature_column.numeric_column(self._num_col, shape=input_shape[self._num_col].as_list()[-1])
        ls_feature_num_col.append(tf_feature_num)
        
        self.processing_layer = tf.keras.layers.DenseFeatures(ls_feature_cat_col + ls_feature_num_col)
        
    def call(self, inputs):
        
        return self.processing_layer(inputs)
    
    def get_config(self):
        
        config = super(DataProcessingLayer, self).get_config()
        config.update({
            'ls_cat_col': self._ls_cat_col,
            'num_col': self._num_col,
            'vocabulary': self._dict_cat_vocab
        })
        
        return config
    
    @classmethod
    def from_config(cls, config):
        
        return cls(**config)
        

In [8]:
def build_data_processing_layer(file_path, dataset, re_build=True):

    if path.exists(file_path) and re_build == False:

        with open(file_path, 'rb') as f:
            serialized_layer = pickle.load(f) 
        layer = tf.keras.layers.deserialize(
            serialized_layer, custom_objects={'DataProcessingLayer': DataProcessingLayer}
        )
        
        print(f'Loaded the saved layer from {file_path}')

        return layer

    else:

        layer = DataProcessingLayer(
            ls_cat_col=CAT_COLUMNS,
            num_col='numeric',
            name='data_processing_layer'
        )
        layer.adapt(data=dataset)
        serialized_layer = tf.keras.layers.serialize(layer)
        with open(file_path, 'wb') as f:
            pickle.dump(serialized_layer, f)
            
        print(f'Saved the layer to {file_path}')
        
        return layer

In [9]:
data_processing_layer = build_data_processing_layer(
    file_path='saved/layer/DataProcessingLayer.pkl',
    dataset=train_dataset,
    re_build=False
)

Loaded the saved layer from saved/layer/DataProcessingLayer.pkl


### 3. ANN Model Building

In [10]:
ann_model_input = {
    'numeric': tf.keras.Input((len(NUM_COLUMNS),), dtype=tf.dtypes.float32, name='numeric_input')
}

for cat_column in CAT_COLUMNS:
    ann_model_input.update({cat_column: tf.keras.Input((1,), dtype=tf.dtypes.string, name=f'{cat_column}_input')})

x = data_processing_layer(ann_model_input)
x = tf.keras.layers.Dense(128, activation="relu", name='hiden_layer_1')(x)
x = tf.keras.layers.Dense(128, activation="relu", name='hiden_layer_2')(x)
ann_model_output = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(x)

ann_model = tf.keras.Model(
    inputs=ann_model_input, 
    outputs=ann_model_output, 
    name='binary_classifer')

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

ann_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=METRICS)

In [11]:
ann_model.summary()

Model: "binary_classifer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cat0_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat1_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat10_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat11_input (InputLayer)        [(None, 1)]          0                                            
___________________________________________________________________________________

### 5. Training

In [12]:
%rm -rf saved/logs

In [13]:
checkpoint_path = "saved/model/cp.ckpt"
checkpoint_dir = path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Create a callback for TensorBoard
logdir = "saved/logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [14]:
model_history = ann_model.fit(
    train_dataset, 
    validation_data=validate_dataset, 
    callbacks=[cp_callback, tensorboard_callback],
    epochs=20,
    verbose=0,
    workers=4)

print(f'Average test loss: {np.average(model_history.history["loss"])})')


Epoch 00001: saving model to saved/model/cp.ckpt

Epoch 00002: saving model to saved/model/cp.ckpt

Epoch 00003: saving model to saved/model/cp.ckpt

Epoch 00004: saving model to saved/model/cp.ckpt

Epoch 00005: saving model to saved/model/cp.ckpt

Epoch 00006: saving model to saved/model/cp.ckpt

Epoch 00007: saving model to saved/model/cp.ckpt

Epoch 00008: saving model to saved/model/cp.ckpt

Epoch 00009: saving model to saved/model/cp.ckpt

Epoch 00010: saving model to saved/model/cp.ckpt

Epoch 00011: saving model to saved/model/cp.ckpt

Epoch 00012: saving model to saved/model/cp.ckpt

Epoch 00013: saving model to saved/model/cp.ckpt

Epoch 00014: saving model to saved/model/cp.ckpt

Epoch 00015: saving model to saved/model/cp.ckpt

Epoch 00016: saving model to saved/model/cp.ckpt

Epoch 00017: saving model to saved/model/cp.ckpt

Epoch 00018: saving model to saved/model/cp.ckpt

Epoch 00019: saving model to saved/model/cp.ckpt

Epoch 00020: saving model to saved/model/cp.ckpt


### 6. Evaluation

#### 6.1 Load testing dataset

In [15]:
testing_dataset = tf.data.experimental.make_csv_dataset(
    file_pattern='data/dac/sample_test.txt', 
    batch_size=200,
    num_epochs=1,
    column_defaults=COLUMN_DEFAULTS,
    column_names=COLUMNS, 
    label_name='lbl', 
    field_delim='\t',
    shuffle=True
).shuffle(10, reshuffle_each_iteration=False)

In [16]:
testing_packed_dataset = testing_dataset.map(FeaturesExtraction(num_col=NUM_COLUMNS))

In [17]:
ann_model.evaluate(testing_packed_dataset, return_dict=True, verbose=0)

{'loss': 0.6737586855888367,
 'tp': 610.0,
 'fp': 0.0,
 'tn': 9513.0,
 'fn': 1828.0,
 'accuracy': 0.8470420837402344,
 'precision': 1.0,
 'recall': 0.2502050995826721,
 'auc': 0.6251025199890137}